In [1]:
script_path = getwd()                                # %exclude_jupyterlab%
script_path                                          # %exclude_jupyterlab%

[1] "/Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna"

In [8]:
system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
        '/sc_clusterprofiler_opt.R --path=/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/seurat/ --outdir="test_dir" --genome="hg38" '),  # %exclude_jupyterlab%
       intern = T) # %exclude_jupyterlab%

character(0)

In [9]:
library(Seurat)
library(clusterProfiler)
library(dplyr)

In [10]:
opt = readRDS("./test_dir/opt.rds")                                 # %exclude_jupyterlab%

In [11]:
opt                                                              

$path
[1] "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/seurat/"

$sampleid
[1] "scRNA"

$genome
[1] "hg38"

$outdir
[1] "test_dir"

$help
[1] FALSE

In [14]:
seuratPath = opt$path
genome     = opt$genome


In [15]:
if (genome == "mm10") {
  library(org.Mm.eg.db)
  orgdb = "org.Mm.eg.db"
  organism = "mmu"
}else if(genome == "hg38"){
  library(org.Hs.eg.db)
  orgdb = "org.Hs.eg.db"
  organism = "hsa"
}

ERROR: Error in library(org.Hs.eg.db): there is no package called ‘org.Hs.eg.db’


In [18]:
library(org.Hs.eg.db)

ERROR: Error in library(org.Hs.eg.db): there is no package called ‘org.Hs.eg.db’


In [17]:
library(org.Mm.eg.db)

ERROR: Error in library(org.Mm.eg.db): there is no package called ‘org.Mm.eg.db’
